### Statistical Learning for Data Science 2 (229352) 
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #10

In [ ]:
import torch
import torch.nn as nn

### Stacking layer with `nn.Sequential`

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(nn.Linear(28*28, 512),
                                          nn.ReLU(),
                                          nn.Linear(512, 512),
                                          nn.ReLU(),
                                          nn.Linear(512, 10))

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_stack(x)
        return logits


model = SimpleNN()

### Model's prediction of an instance

In [ ]:
x = torch.rand(28, 28, 1)

x_us = x.unsqueeze(axis=0)
print(x_us.shape)

model(x_us)

# Image classification with LeNet-5

![lenet5](http://d2l.ai/_images/lenet.svg)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt

In [ ]:
# Hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 5

# Classification
NUM_CLASSES = 10

# CIFAR10 dataset

Classify (32x32x3) images into 10 classes

Here, `train_dataset` and `val_dataset` are `(image, label)` generators

In [ ]:
train_dataset = datasets.CIFAR10(root='data',
                                 train=True,
                                 transform=transforms.ToTensor(),
                                 download=True)

val_dataset = datasets.CIFAR10(root='data',
                                train=False,
                                transform=transforms.ToTensor())

Look at an image in the dataset

In [ ]:
image, label = train_dataset[1]

print(image.shape)

plt.imshow(image.permute(1, 2, 0))

# Data augmentation

![augmentation](https://miro.medium.com/max/700/0*LR1ZQucYW96prDte)

See more transformations in [Pytorch documentation](https://pytorch.org/vision/0.9/transforms.html)

In [ ]:
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomResizedCrop((32, 32),
                                                                   scale=(0.7,0.8),
                                                                   ratio=(0.6,0.7)),
                                     transforms.ToTensor()
                                     ])

train_dataset = datasets.CIFAR10(root='data',
                                 train=True,
                                 transform=train_transform,
                                 download=True)

val_dataset = datasets.CIFAR10(root='data',
                                train=False,
                                transform=transforms.ToTensor())

In [ ]:
image, labels = train_dataset[6]
plt.imshow(image.permute(1,2,0))

Use `DataLoader` to split the dataset into minibatches.

In [ ]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=2,
                          shuffle=True)

val_loader = DataLoader(dataset=val_dataset,
                         batch_size=BATCH_SIZE,
                         num_workers=2,
                         shuffle=False)


Check the size of each minibatch

In [ ]:
for images, labels in train_loader:
  print(images.shape)
  break

# LeNet5

![lenet5](http://d2l.ai/_images/lenet.svg)

In [ ]:
class LeNet5(nn.Module):

    def __init__(self, num_classes):
        super(LeNet5, self).__init__()

        self.num_classes = num_classes

        self.features = nn.Sequential(
            # Input shape = 28 x 28 x 3
            nn.Conv2d(in_channels=3, out_channels=6*3, kernel_size=5),
            # Output shape = 28 - (5 - 1) = 24
            nn.Dropout(0.2),
            nn.BatchNorm2d(6*3),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=3),
            # shape = 24/3 = 8
            nn.Conv2d(in_channels=6*3, out_channels=16*3, kernel_size=5),
            # shape = 8 - (5 - 1) = 4
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2)
            # shape = 4/2 = 2
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16*2*2*3, 3*120),
            nn.Tanh(),
            nn.Linear(3*120, 3*84),
            nn.Tanh(),
            nn.Linear(3*84, self.num_classes)
        )


    def forward(self, x):
      x = self.features(x)
      logits = self.classifier(x)
      return logits

model = LeNet5(NUM_CLASSES)

Apply the model on an image

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(NUM_EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(val_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306655  [    0/50000]
loss: 1.612821  [12800/50000]
loss: 1.499798  [25600/50000]
loss: 1.417957  [38400/50000]
Test Error: 
 Accuracy: 47.8%, Avg loss: 1.432063 

Epoch 2
-------------------------------
loss: 1.521928  [    0/50000]
loss: 1.587631  [12800/50000]
loss: 1.336206  [25600/50000]
loss: 1.337718  [38400/50000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 1.335955 

Epoch 3
-------------------------------
loss: 1.227350  [    0/50000]
loss: 1.276722  [12800/50000]
loss: 1.187620  [25600/50000]
loss: 1.143427  [38400/50000]
Test Error: 
 Accuracy: 55.5%, Avg loss: 1.251594 

Epoch 4
-------------------------------
loss: 1.198739  [    0/50000]
loss: 1.228100  [12800/50000]
loss: 1.153681  [25600/50000]
loss: 1.302835  [38400/50000]
Test Error: 
 Accuracy: 57.3%, Avg loss: 1.205092 

Epoch 5
-------------------------------
loss: 1.148103  [    0/50000]
loss: 0.987750  [12800/50000]
loss: 1.215715  [25600/50000]
loss: 1.129617  [38400

In [ ]:
image, label = train_dataset[1]

print("Label =", label)

print("Prediction =", model(image.unsqueeze(axis=0)))

Label = 9
Prediction = tensor([[-0.1624,  1.3433, -1.1425, -2.4824, -1.1832, -0.5600, -3.7437,  2.0615,
         -2.5818,  7.4315]], grad_fn=<AddmmBackward0>)


# Exercise

1. Modify the model to have at least **3 convolution layers** and **3 Max-pooling layers** (and possibly some batch normalization and some dropout layers)
2. Change the activation function from Tanh to other functions (see [Pytorch documentation](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity))
3. Your model must have more than 69% accuracy on validation set.